In [ ]:
# to do: fill NaN using the values from the last vintage date

In [5]:
import pathlib, warnings
import pandas as pd
import numpy as np
import datetime as datetime

In [ ]:
df_descriptions = pd.read_csv('raw_variables_descriptions.csv')

In [ ]:
data_path = pathlib.Path('raw_variables')
assert data_path.exists()

In [67]:
vintage_date = '2011-04-01' # until the end of day
start_quarter = '1981Q1' # inclusive
end_quarter = '2010Q2' # inclusive

vintage_date = pd.to_datetime(vintage_date)
start_observation = pd.to_datetime(start_quarter).to_period('Q').start_time
end_observation = pd.to_datetime(end_quarter).to_period('Q').end_time

if vintage_date < end_observation:
    end_observation = vintage_date

fillna_from_last_vintage = False
variables = {
    'raw_variables': ['DGS10'],
    'observed_variables': [],
}

In [68]:
index = -1

for data_file in set(data_path.glob('*.*')):

    variable = data_file.stem
    to_merge = False

    if variable in variables['raw_variables']:

        data_set = pd.read_csv(data_file, index_col=0)
        data_set.index = pd.to_datetime(data_set.index)
        frequency = df_descriptions[df_descriptions['id']==variable]['frequency_short'].values[0]

        # choose data within observation period
        observation_dates = data_set.index.map(lambda x: start_observation <= x <= end_observation)
        if observation_dates.values.sum() == 0:
            warnings.warn(f'\n{variable} has no value within the observation period you choose for any vintage date!\n')

        # choose vintage date
        vintage_column = ''
        if frequency != 'D':
            for column in data_set.columns.values:
                if int(vintage_date.strftime('%Y%m%d')) >= int(column[-8:]):
                    vintage_column = column
                else:
                    break
            if vintage_column == '':
                warnings.warn(f'\nFor {variable}, the vintage date you choose is out of bound!\n')
        else:
            vintage_column = data_set.columns.values[-1]

        # leave only required data
        # for daily and monthly data, take average over the quarter
        if observation_dates.values.sum() > 0 and vintage_column != '':

            data_set['quarter'] = data_set.index.to_period('Q').values
            data_set = data_set[observation_dates][[vintage_column, 'quarter']].copy()

            if np.sum(~np.isnan(data_set[vintage_column].values)) == 0:
                warnings.warn(f'\n{variable} has no value in the observation period and vintage date you choose.\n')
            else:
                index += 1
                to_merge = True
                data_to_merge = data_set.groupby('quarter').mean()
    
    if to_merge == True:
        if index == 0:
            data_output = data_to_merge.copy()
        else:
            data_output = pd.merge(data_output, data_to_merge, how='outer', left_index=True, right_index=True, sort=True)

In [69]:
data_output.to_csv(f"data_{vintage_date.strftime('%Y%m%d')}.csv")